<a href="https://colab.research.google.com/github/yjh111172/medical-data-analysis/blob/main/07_Tensorflow_using_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# for문 연산

- 피어슨 상관계수 : -1부터 1까지 중 1에 가까울수록 양의 상관 관계가 강하고, -1에 가까울수록 음의 상관 관계가 강함, 0에 가까울수록 상관 관계가 약함
- 유전자형 데이터(SNP)와 유전자 발현 데이터(Expression) 간의 연관성 찾기
- SNP와 유전자 발현 간의 상관계수를 계산하고, 그 중에서 가장 높은 상관계수의 제곱 반환

In [ ]:
from scipy.stats import pearsonr
import numpy as np

In [ ]:
def max_corr(X, Y) :
  maxr2 = 0
  for x in X :
    for y in Y :
      r, p = pearsonr(x, y)
      if r*r > maxr2 :
        maxr2 = r*r
  return maxr2

In [ ]:
snp = np.random.rand(1000, 100) # 1000개의 샘플과 100개의 SNP를 나타내는 2차원 배열
expr = np.random.rand(200, 100) # 200개의 유전자 발현 수준과 100개의 개체을 나타내는 2차원 배열

In [ ]:
# max_corr(snp, expr) # np.float64(0.19126230894173835) , 10분 이상 소요

In [ ]:
# 10000 * 2000 쌍의 상관 계수 구하기
snp = np.random.rand(10000, 100)
expr = np.random.rand(2000, 100)

In [ ]:
# timeit -r1 -n1 max_corr(snp, expr) 30분 이상 소요

In [ ]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.30
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt
                           aes xsave avx f16c rdrand hypervisor 

# numpy 연산
- 루프 성능 개선을 위해 numpy 연산 사용
- r^2 = (공분산)^2 / (X의 분산 * Y의 분산)

In [ ]:
def max_corr_np(X, Y) :
  E_X = X.mean(axis=1, keepdims = True)
  E_Y = Y.mean(axis=1, keepdims = True)
  sigma_XY = (X - E_X) @ (Y - E_Y).T # @는 행렬곱, T는 전치행렬, 공분산 계산
  sumsq_X = ((X - E_X)**2).sum(axis=1, keepdims = True) # 분산
  sumsq_Y = ((Y - E_Y)**2).sum(axis=1) # 분산
  r2 = sigma_XY**2 / sumsq_X / sumsq_Y
  return r2.max()

In [ ]:
timeit -r1 -n1 max_corr_np(snp, expr)

706 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Tensorflow
- 텐서플로우를 통한 연산 수행
- numpy 연산 형태와 비슷

In [ ]:
import tensorflow as tf

In [ ]:
def max_corr_tf(X, Y) :
  E_X = tf.reduce_mean(X, axis=1, keepdims = True)
  E_Y = tf.reduce_mean(Y, axis=1, keepdims = True)
  sigma_XY = tf.matmul((X - E_X), (Y - E_Y), transpose_b=True)   # tf.matmul: 두 행렬의 곱, Y의 행렬을 전치하여 곱셈 수행
  sumsq_X = tf.reduce_sum((X - E_X)**2, axis=1, keepdims = True)
  sumsq_Y = tf.reduce_sum((Y - E_Y)**2, axis=1)
  r2 = sigma_XY**2 / sumsq_X / sumsq_Y
  max_r2 = tf.reduce_max(r2)

  return max_r2.numpy()

In [ ]:
timeit -r1 -n1 max_corr_tf(snp, expr)

126 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
print(max_corr_np(snp, expr))
print(max_corr_tf(snp, expr))

0.2753590654822197
0.27535906548221983


# Keras 실습

In [ ]:
from keras.models import Sequential # 입력층, 은닉층, 출력층으로 구성
from keras.layers import Dense # 각 뉴런이 이전 층의 모든 뉴런과 연결

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim=256, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,057 (144.75 KB)

 Trainable params: 37,057 (144.75 KB)

 Non-trainable params: 0 (0.00 B)